In [1]:
import pandas as pd
import numpy as np
import lightgbm
#from sklearn.model_selection import train_test_split

In [2]:
train_csv = pd.read_csv('C:/Users/bruce/Desktop/DMT_Assignment2/training_set_VU_DM.csv')

In [3]:
y= np.where((train_csv['booking_bool'] == 1), 2, np.where((train_csv['click_bool'] ==1) & (train_csv['booking_bool'] == 0), 1, 0))
train_csv["date_time"] = pd.to_datetime(train_csv["date_time"])
train_csv["day"] = train_csv["date_time"].dt.day
train_csv["month"] = train_csv["date_time"].dt.month
train_csv.drop(labels=["date_time","click_bool", "booking_bool", "gross_bookings_usd"], axis=1, inplace=True)
ignore_columns = ["visitor_hist_adr_usd", "visitor_hist_starrating", "srch_query_affinity_score"]
missing_perc = train_csv.isnull().mean() * 100
keep_cols = missing_perc[missing_perc <= 90]  # Ignore columns with more than 90% missing data
keep_cols = keep_cols.index.union(ignore_columns)  # Add the specified columns to the keep list
pre_feature = train_csv[keep_cols]


In [4]:
competitor_columns = [
    "comp2_inv",
    "comp2_rate",
    "comp2_rate_percent_diff",
    "comp3_inv",
    "comp3_rate",
    "comp5_inv",
    "comp5_rate",
    "comp5_rate_percent_diff",
    "comp8_inv",
    "comp8_rate",
    "comp8_rate_percent_diff",
]

pre_feature[competitor_columns] = pre_feature[competitor_columns].fillna(0)
pre_feature["prop_location_score2"].fillna(pre_feature["prop_location_score2"].min(), inplace=True)
pre_feature["prop_review_score"].fillna(pre_feature["prop_review_score"].min(), inplace=True)


C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\3638886222.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_feature[competitor_columns] = pre_feature[competitor_columns].fillna(0)
C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\3638886222.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_feature["prop_location_score2"].fillna(pre_feature["prop_location_score2"].min(), inplace=True)
C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\3638886222.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

In [5]:
mean_prop_usd = pre_feature.groupby("prop_id")["price_usd"].transform("mean")
std_prop_usd = pre_feature.groupby("prop_id")["price_usd"].transform("std")
pre_feature["normalized_price_usd_per_prop_id"] = (pre_feature["price_usd"] - mean_prop_usd) / std_prop_usd

mean_srch_usd = pre_feature.groupby("srch_id")["price_usd"].transform("mean")
std_srch_usd = pre_feature.groupby("srch_id")["price_usd"].transform("std")
pre_feature["normalized_price_usd_per_srch_id"] = (pre_feature["price_usd"] - mean_srch_usd) / std_srch_usd

mean_srch_prop_starrating = pre_feature.groupby("srch_id")["prop_starrating"].transform("mean")
std_srch_prop_starrating = pre_feature.groupby("srch_id")["prop_starrating"].transform("std")
pre_feature["normalized_prop_starrating_per_srch_id"] = (pre_feature["prop_starrating"] - mean_srch_prop_starrating) / std_srch_prop_starrating

mean_srch_prop_location_score2 = pre_feature.groupby("srch_id")["prop_location_score2"].transform("mean")
std_srch_prop_location_score2 = pre_feature.groupby("srch_id")["prop_location_score2"].transform("std")
pre_feature["normalized_prop_location_score2_per_srch_id"] = (pre_feature["prop_location_score2"] - mean_srch_prop_location_score2) / std_srch_prop_location_score2

mean_srch_prop_location_score1 = pre_feature.groupby("srch_id")["prop_location_score1"].transform("mean")
std_srch_prop_location_score1 = pre_feature.groupby("srch_id")["prop_location_score1"].transform("std")
pre_feature["normalized_prop_location_score1_per_srch_id"] = (pre_feature["prop_location_score1"] - mean_srch_prop_location_score1) / std_srch_prop_location_score1

mean_srch_prop_review_score = pre_feature.groupby("srch_id")["prop_review_score"].transform("mean")
std_srch_prop_review_score = pre_feature.groupby("srch_id")["prop_review_score"].transform("std")
pre_feature["normalized_prop_review_score_per_srch_id"] = (pre_feature["prop_review_score"] - mean_srch_prop_review_score) / std_srch_prop_review_score


C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\3624791867.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_feature["normalized_price_usd_per_prop_id"] = (pre_feature["price_usd"] - mean_prop_usd) / std_prop_usd
C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\3624791867.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_feature["normalized_price_usd_per_srch_id"] = (pre_feature["price_usd"] - mean_srch_usd) / std_srch_usd
C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\3624791867.py:11: Se

In [6]:
cleaned_train = pre_feature.copy()

In [7]:
# Training data split with feature creation (need to check)

#x1, x2, y1, y2 = train_test_split(cleaned_train, y, test_size=0.2, random_state=42)
train_size = int(0.8 * len(cleaned_train))
x1 = cleaned_train.iloc[:train_size]
x2 = cleaned_train.iloc[train_size:]
y1 = y[:train_size]
y2 = y[train_size:]
srch_id_dest_id_dict = x1.loc[x1["random_bool"] == 0]

# estimated position calculation
srch_id_dest_id_dict = x1.loc[x1["random_bool"] == 0]
srch_id_dest_id_dict = x1.groupby(["srch_destination_id", "prop_id"]).agg(
    {"position": "mean"}
)
srch_id_dest_id_dict = srch_id_dest_id_dict.rename(
    index=str, columns={"position": "estimated_position"}
).reset_index()
srch_id_dest_id_dict["srch_destination_id"] = (
    srch_id_dest_id_dict["srch_destination_id"].astype(str).astype(int)
)
srch_id_dest_id_dict["prop_id"] = (
    srch_id_dest_id_dict["prop_id"].astype(str).astype(int)
)
srch_id_dest_id_dict["estimated_position"] = (
    1 / srch_id_dest_id_dict["estimated_position"]
)

x1 = x1.merge(srch_id_dest_id_dict, how="left", on=["srch_destination_id", "prop_id"])
x2 = x2.merge(srch_id_dest_id_dict, how="left", on=["srch_destination_id", "prop_id"])

groups = x1["srch_id"].value_counts(sort=False).sort_index()
eval_groups = x2["srch_id"].value_counts(sort=False).sort_index()

x1.drop(labels=["srch_id", "prop_id", "position", "random_bool"], axis=1, inplace=True)
x2.drop(labels=["srch_id", "prop_id", "position", "random_bool"], axis=1, inplace=True)

In [8]:
# feature creation for LambdaMart to handle categorical variables

categorical_features = [
        "day",
        "month",
        "prop_country_id",
        "site_id",
        "visitor_location_country_id",
    ]
categorical_features = [c for c in categorical_features if c in x1.columns.values]
categorical_features_numbers = [x1.columns.get_loc(x) for x in categorical_features]

In [9]:
clf = lightgbm.LGBMRanker(
        objective="lambdarank",
        metric="ndcg",
        n_estimators=2000,
        learning_rate=0.1,
        max_position=5,
        label_gain=[0, 1, 5],
        seed=70,
        boosting="dart",
    )

clf.fit(
        x1,
        y1,
        eval_set=[(x1, y1), (x2, y2)],
        eval_group=[groups, eval_groups],
        group=groups,
        eval_at=5,
        verbose=200,    
        categorical_feature=categorical_features_numbers,
    )

c:\Users\bruce\anaconda3\envs\troll\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\bruce\anaconda3\envs\troll\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\bruce\anaconda3\envs\troll\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [11, 12, 17, 23, 34]
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Unknown parameter: max_position
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\Users\bruce\anaconda3\envs\troll\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\Users\bruce\anaconda3\envs\troll\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[200]	training's ndcg@5: 0.413153	valid_1's ndcg@5: 0.390267
[400]	training's ndcg@5: 0.426647	valid_1's ndcg@5: 0.395136
[600]	training's ndcg@5: 0.437198	valid_1's ndcg@5: 0.397335
[800]	training's ndcg@5: 0.448312	valid_1's ndcg@5: 0.398567
[1000]	training's ndcg@5: 0.457716	valid_1's ndcg@5: 0.399832
[1200]	training's ndcg@5: 0.467834	valid_1's ndcg@5: 0.399481
[1400]	training's ndcg@5: 0.477055	valid_1's ndcg@5: 0.401006
[1600]	training's ndcg@5: 0.48436	valid_1's ndcg@5: 0.400477
[1800]	training's ndcg@5: 0.4922	valid_1's ndcg@5: 0.401091
[2000]	training's ndcg@5: 0.498346	valid_1's ndcg@5: 0.400853


LGBMRanker(boosting='dart', label_gain=[0, 1, 5], max_position=5, metric='ndcg',
           n_estimators=2000, objective='lambdarank', seed=70)

In [10]:
test_csv = pd.read_csv('C:/Users/bruce/Desktop/DMT_Assignment2/test_set_VU_DM.csv')

In [11]:
# Test data cleaning 
test_csv["date_time"] = pd.to_datetime(test_csv["date_time"])
test_csv["day"] = test_csv["date_time"].dt.day
test_csv["month"] = test_csv["date_time"].dt.month
test_csv.drop(labels=["date_time"], axis=1, inplace=True)
ignore_columns_test = ["visitor_hist_adr_usd", "visitor_hist_starrating", "srch_query_affinity_score"]
missing_perc_test = test_csv.isnull().mean() * 100
keep_cols_test = missing_perc_test[missing_perc_test <= 90]  
keep_cols_test = keep_cols_test.index.union(ignore_columns_test)  
pre_feature_test = test_csv[keep_cols_test]

In [12]:
competitor_columns = [
    "comp2_inv",
    "comp2_rate",
    "comp2_rate_percent_diff",
    "comp3_inv",
    "comp3_rate",
    "comp5_inv",
    "comp5_rate",
    "comp5_rate_percent_diff",
    "comp8_inv",
    "comp8_rate",
    "comp8_rate_percent_diff",
]

pre_feature_test[competitor_columns] = pre_feature_test[competitor_columns].fillna(0)
pre_feature_test["prop_location_score2"].fillna(pre_feature_test["prop_location_score2"].min(), inplace=True)
pre_feature_test["prop_review_score"].fillna(pre_feature_test["prop_review_score"].min(), inplace=True)

C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\572512566.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_feature_test[competitor_columns] = pre_feature_test[competitor_columns].fillna(0)
C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\572512566.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_feature_test["prop_location_score2"].fillna(pre_feature_test["prop_location_score2"].min(), inplace=True)
C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\572512566.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [13]:
mean_prop_usd = pre_feature_test.groupby("prop_id")["price_usd"].transform("mean")
std_prop_usd = pre_feature_test.groupby("prop_id")["price_usd"].transform("std")
pre_feature_test["normalized_price_usd_per_prop_id"] = (pre_feature_test["price_usd"] - mean_prop_usd) / std_prop_usd

mean_srch_usd = pre_feature_test.groupby("srch_id")["price_usd"].transform("mean")
std_srch_usd = pre_feature_test.groupby("srch_id")["price_usd"].transform("std")
pre_feature_test["normalized_price_usd_per_srch_id"] = (pre_feature_test["price_usd"] - mean_srch_usd) / std_srch_usd

mean_srch_prop_starrating = pre_feature_test.groupby("srch_id")["prop_starrating"].transform("mean")
std_srch_prop_starrating = pre_feature_test.groupby("srch_id")["prop_starrating"].transform("std")
pre_feature_test["normalized_prop_starrating_per_srch_id"] = (pre_feature_test["prop_starrating"] - mean_srch_prop_starrating) / std_srch_prop_starrating

mean_srch_prop_location_score2 = pre_feature_test.groupby("srch_id")["prop_location_score2"].transform("mean")
std_srch_prop_location_score2 = pre_feature_test.groupby("srch_id")["prop_location_score2"].transform("std")
pre_feature_test["normalized_prop_location_score2_per_srch_id"] = (pre_feature_test["prop_location_score2"] - mean_srch_prop_location_score2) / std_srch_prop_location_score2

mean_srch_prop_location_score1 = pre_feature_test.groupby("srch_id")["prop_location_score1"].transform("mean")
std_srch_prop_location_score1 = pre_feature_test.groupby("srch_id")["prop_location_score1"].transform("std")
pre_feature_test["normalized_prop_location_score1_per_srch_id"] = (pre_feature_test["prop_location_score1"] - mean_srch_prop_location_score1) / std_srch_prop_location_score1

mean_srch_prop_review_score = pre_feature_test.groupby("srch_id")["prop_review_score"].transform("mean")
std_srch_prop_review_score = pre_feature_test.groupby("srch_id")["prop_review_score"].transform("std")
pre_feature_test["normalized_prop_review_score_per_srch_id"] = (pre_feature_test["prop_review_score"] - mean_srch_prop_review_score) / std_srch_prop_review_score


C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\3132412966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_feature_test["normalized_price_usd_per_prop_id"] = (pre_feature_test["price_usd"] - mean_prop_usd) / std_prop_usd
C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\3132412966.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_feature_test["normalized_price_usd_per_srch_id"] = (pre_feature_test["price_usd"] - mean_srch_usd) / std_srch_usd
C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\

In [14]:
cleaned_test = pre_feature_test.copy()

In [15]:
cleaned_test = cleaned_test.merge(srch_id_dest_id_dict, how="left", on=["srch_destination_id", "prop_id"])
test_data_srch_id_prop_id = cleaned_test[["srch_id", "prop_id"]]
cleaned_test.drop(labels=["srch_id", "prop_id", "random_bool"], axis=1, inplace=True)

categorical_features_test = [
        "day",
        "month",
        "prop_country_id",
        "site_id",
        "visitor_location_country_id",
    ]
categorical_features_test = [c for c in categorical_features_test if c in cleaned_test.columns.values]
categorical_features_test_numbers = [x1.columns.get_loc(x) for x in categorical_features_test]


In [16]:
predictions = clf.predict(cleaned_test, categorical_feature=categorical_features_numbers)
test_data_srch_id_prop_id["prediction"] = predictions
test_data_srch_id_prop_id = test_data_srch_id_prop_id.sort_values(
        ["srch_id", "prediction"], ascending=False
    )

test_data_srch_id_prop_id[["srch_id", "prop_id"]].to_csv('submission.csv',index=False)

C:\Users\bruce\AppData\Local\Temp\ipykernel_17440\2191333886.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_srch_id_prop_id["prediction"] = predictions
